<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain --progress-bar off
!pip install -qU langchainhub --progress-bar off
!pip install -qU duckduckgo-search --progress-bar off
!pip install -qU fireworks-ai --progress-bar off
!pip install -qU openai --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
# @title load fireworks API key
#connect to google drive
from google.colab import drive
import json
import os


drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']


Mounted at /content/drive


In [ ]:
import openai
# This is required to make it work for old version of openai < 1
openai.api_base = "https://api.fireworks.ai/inference/v1"
openai.api_key = env['fireworks.ai']['apiKey']

In [ ]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
# Turn this on only if you want to debug other wise it's hard to see the conversations.
set_debug(True)

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import tool
from typing import Literal
from typing_extensions import Annotated

TemperatureUnitSymbol = Literal["celcius", "fahrenheit"]

class GetCurrentWeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: TemperatureUnitSymbol = Field(description="The temperature unit value")

@tool(args_schema=GetCurrentWeatherInput)
def get_current_weather(location, unit = "fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return f"Current location in {location} is 72 {unit}"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function


functions = [get_current_weather]
tools = [{ "type": "function", "function": format_tool_to_openai_function(f)} for f in functions]

# Initialize a Fireworks chat model
# For function calling we cannot use ChatFireworks integration as it doesn't properly pass functions
llm = ChatOpenAI(model="accounts/fireworks/models/fw-function-call-34b-v0",
                 openai_api_key=env['fireworks.ai']['apiKey'],
                 openai_api_base="https://api.fireworks.ai/inference/v1",
                 # verbose=True,
                 temperature= 0, max_tokens= 1024,
                 model_kwargs={ "tools":tools }
                )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.schema.output_parser import BaseLLMOutputParser
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnableLambda
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    BaseMessageChunk,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

def format_agent_scratchpad_from_intermediate_steps(x):
  return format_to_openai_tool_messages(x["intermediate_steps"])

# having error when calling convert_message_to_dict from langchain (https://github.com/langchain-ai/langchain/blob/4c47f39fcb539fdeff6dd6d9b1f483cd9a1af69b/libs/community/langchain_community/adapters/openai.py#L104C5-L104C28)
# because when we use tool we are also submitting that to openai (with 'name' in payload which fireworks is not supported yet).
# By default langchain also send functions/tools as role to openai.
# fireworks doesn't support that so we are going to adjust the prompt
def prepare_prompt_for_llm(x):
  messages = []
  print('prepare_prompt_for_llm')
  for message in x.messages:
    if isinstance(message, ToolMessage):
      # remove the name as we don't want to pass that in fireworks.
      message = ToolMessage(content=message.content, tool_call_id=message.tool_call_id)

    messages.append(message)


  print(messages)
  return messages


agent = {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_agent_scratchpad_from_intermediate_steps(x),
    } | prompt | RunnableLambda(prepare_prompt_for_llm) | llm | OpenAIToolsAgentOutputParser()


agent_executor = AgentExecutor(agent=agent, tools=[get_current_weather], verbose=True)


agent_executor.invoke({"input": "what is the weather is sf?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is the weather is sf?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "what is the weather is sf?"
}
[chain/start] [1:chain:AgentExecutor >

{'input': 'what is the weather is sf?',
 'output': 'The current weather in sf is 72 fahrenheit. '}

In [ ]:
import fireworks.client
fireworks.client.api_key = env['fireworks.ai']['apiKey']
fireworks.client.ChatCompletion.create(
  model="accounts/fireworks/models/fw-function-call-34b-v0",
  messages=[
    {
      "role": "system",
      "content": "You are very powerful assistant, but don't know current events"
    },
    {
      "role": "user",
      "content": "what is the weather is sf?"
    },
    {
      "role": "assistant",
      "content": ""
    },
    {
      "role": "tool",
      "content": "Current location in sf is 72 fahrenheit",
      "tool_call_id": "call_ByENVAlPvpTBvqqrr0vTAfCv"
    }
  ],
  n=1,
  max_tokens=1000,
  temperature=0.9,
  stop=[],
)

ChatCompletionResponse(id='cmpl-2e547d648a6097e91ad7856d', object='chat.completion', created=1704706988, model='accounts/fireworks/models/fw-function-call-34b-v0', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='The current temperature in San Francisco is 72 degrees Fahrenheit. ', tool_calls=None, tool_call_id=None, function=None), finish_reason='stop', logprobs=None)], usage=UsageInfo(prompt_tokens=60, total_tokens=77, completion_tokens=17))